In [2]:
import pickle
from sklearn.feature_extraction import DictVectorizer
import json
from google.colab import drive
drive.mount('/content/drive/')
!pip install transformers==3
import os
import json
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import classification_report
import re
import transformers
from transformers import AutoModel, BertTokenizerFast
import torch
import torch.nn as nn
import pickle


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
demo_folder='ottawa'

In [5]:
files='/content/drive/MyDrive/CAPSTONE PROJECT - PRAJNA SIRISHA SUKANYA/demo/Tweet_data/'+demo_folder
folds = sorted(os.listdir(files))#id
new_path = os.path.join(files, folds[0])
ids = sorted(os.listdir(new_path)) 
newfolds = [i for i in ids if i[0] != '.']
id_data = newfolds
text = ''
full_replies_text = []
full_tweet_text = []
source_path = os.path.join(new_path,'source-tweet')
source_file_name = sorted(os.listdir(source_path)) 
name = [i for i in source_file_name if i[0] != '.']
source_file = open(os.path.join(source_path, name[0]))
print(os.path.join(source_path, name[0]))
for line in source_file:
  source_tweet = json.loads(line)
  text = text+source_tweet['text']
  print("SOURCE-TWEET:",source_tweet['text'])
full_tweet_text.append(text)
final_path = os.path.join(new_path,'replies')
tweets = sorted(os.listdir(final_path))
for tweet in tweets:
  with open(os.path.join(final_path, tweet)) as f:
      for line in f:
          tw = json.loads(line)
          print("REPLY:",tw['text'])
          text = text+tw['text']
full_replies_text.append(text)

/content/drive/MyDrive/CAPSTONE PROJECT - PRAJNA SIRISHA SUKANYA/demo/Tweet_data/ottawa/525032872647065600/source-tweet/525032872647065600.json
SOURCE-TWEET: BREAKING: Michael Zehaf-Bebeau had been designated "high-risk traveller" by CDN govt, which confiscated his passport http://t.co/dPeMQ78jzm
REPLY: @20committee @BlogsofWar How come we have to pussy foot around terrorism for hours before calling what it is.
REPLY: @20committee Pathetic.
REPLY: @20committee @kshaidle maybe we should start revoking passports AFTER they leave...
REPLY: If Michael Zehaf-Bebeau had his passport confiscated doesn't it make sense to track his movements - was this done? @20committee @BlogsofWar
REPLY: @aenk963 @BlogsofWar yes, but CSIS can't watch that many 24/7
REPLY: Yes:) except in Australia there's a discussion to let 'sympathizers' reenter OZ to rehabilitate! WTF! @MarkOkanagan @20committee @kshaidle
REPLY: @20committee @sqwerin and @BarackObama still refuses to call it terrorism!
REPLY: micro chip th

In [6]:

def drop_emoji(x):
    x= re.sub(r"http\S+", "", x)
    x=re.sub(r"[^\w\d\!\?\s]+",'',x)
    drop_list = []
    for i,_x in enumerate(x):
        if ord(_x) >= 128:
            drop_list.append(i)
    new_x = []
    for i,_x in enumerate(x):
        if i not in drop_list:
            new_x.append(_x)
    return ''.join(new_x)
tweets = list(map(drop_emoji,full_replies_text))
# print(tweets)

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"

bert = AutoModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')




# print(tweets)
file1='/content/drive/MyDrive/CAPSTONE PROJECT - PRAJNA SIRISHA SUKANYA/demo/Tweetstructs/ottawa.json'
f=open(file1)
tweets=list(map(drop_emoji,full_tweet_text))
tweet_dict=json.load(f)
tweets_dict=[]
tweets_dict.append(tweet_dict)
v = DictVectorizer(sparse=False)
tweets_x=v.fit_transform(tweets_dict)

# print(tweets)
tokens_test = tokenizer.batch_encode_plus(
    tweets,
    max_length = 100,
    pad_to_max_length=True,
    truncation=False,
    padding=True
)

test_seq = torch.tensor(tokens_test['input_ids'])
test_mask = torch.tensor(tokens_test['attention_mask'])
# test_y = torch.tensor(test_labels)


for param in bert.parameters():
    param.requires_grad = False

class BERT_Arch(nn.Module):
    def __init__(self, bert):
        
        super(BERT_Arch, self).__init__()

        self.bert = bert 
        
        # dropout layer
        self.dropout = nn.Dropout(0.1)
        
        # relu activation function
        self.relu =  nn.ReLU()

        # dense layer 1
        self.fc1 = nn.Linear(768,512)
        
        # dense layer 2 (Output layer)
        self.fc2 = nn.Linear(512,2)

        #softmax activation function
        self.softmax = nn.LogSoftmax(dim=1)

    #define the forward pass
    def forward(self, sent_id, mask):

        #pass the inputs to the model  
        _, cls_hs = self.bert(sent_id, attention_mask=mask)
        
        x = self.fc1(cls_hs)

        x = self.relu(x)

        x = self.dropout(x)

        # output layer
        x = self.fc2(x)
        
        # apply softmax activation
        x = self.softmax(x)

        return x


classes=["Non-Rumour","Rumour"]
model = BERT_Arch(bert)
path = '/content/drive/MyDrive/CAPSTONE PROJECT - PRAJNA SIRISHA SUKANYA/code/bert_saved_weights.pt'
model.load_state_dict(torch.load(path),strict=False)

with torch.no_grad():
  # preds = model(test_seq.to(device), test_mask.to(device))
  pred_logit=model(test_seq.to(device), test_mask.to(device))
  results = torch.softmax(pred_logit, dim=1).tolist()[0]
  # print(pred_logit)
  preds = pred_logit.detach().cpu().numpy()
  # print(preds)

# for i in range(len(classes)):
    # print(f"{classes[i]}: {round(results[i] * 100)}%")

prob_nr=results[0]
prob_r=results[1]
pred_text=""
preds =list(np.argmax(preds, axis = 1))
# print(preds)
for i in range(len(preds)):
    if(preds[i]==0):
        preds[i]="Non-Rumour"
        pred_text=preds[i]
        
    else:
        preds[i]="Rumour"
        pred_text=preds[i]
    
# print(classification_report(test_y, preds))
message ="Detection via transformer (text-based) model"
print(message.center(40,'~'))
for i in range(len(full_replies_text)):
    print(i+1,full_replies_text,'\n',preds[i].center(40,' '))

diff1=abs(prob_nr-prob_r)
prob_nr_text=prob_nr
prob_r_text=prob_r

Detection via transformer (text-based) model
1 ['BREAKING: Michael Zehaf-Bebeau had been designated "high-risk traveller" by CDN govt, which confiscated his passport http://t.co/dPeMQ78jzm@20committee @BlogsofWar How come we have to pussy foot around terrorism for hours before calling what it is.@20committee Pathetic.@20committee @kshaidle maybe we should start revoking passports AFTER they leave...If Michael Zehaf-Bebeau had his passport confiscated doesn\'t it make sense to track his movements - was this done? @20committee @BlogsofWar@aenk963 @BlogsofWar yes, but CSIS can\'t watch that many 24/7Yes:) except in Australia there\'s a discussion to let \'sympathizers\' reenter OZ to rehabilitate! WTF! @MarkOkanagan @20committee @kshaidle@20committee @sqwerin and @BarackObama still refuses to call it terrorism!micro chip them - Just an uncivil thought :) @20committee @BlogsofWar@20committee @AresXtra They must put high risk travelers in civilian detaine camps to prevent attacks.US does th

In [8]:
filename = '/content/drive/MyDrive/CAPSTONE PROJECT - PRAJNA SIRISHA SUKANYA/code/rf_tree (1).pkl'
loaded_model = pickle.load(open(filename, 'rb'))
# print(tweets_x)
preds=loaded_model.predict(tweets_x)
y_pred=loaded_model.predict_proba(tweets_x)
# print(y_pred)
prob_nr=y_pred[0][0]
prob_r=y_pred[0][1]
pred_graph=""
message ="Detection via Graph model"
print(message.center(40,'~'))
for pred in preds:
  if(pred==0):
    pred_graph="Non-Rumour"
  else:
    pred_graph="Rumour"
print(pred_graph.center(40,' '))
diff2=abs(prob_nr-prob_r)
prob_nr_graph=prob_nr
prob_r_graph=prob_r

~~~~~~~Detection via Graph model~~~~~~~~
                 Rumour                 


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.23.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.23.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [9]:
sum1=(-1*(prob_nr_text)+prob_r_text)*(0.71)
sum2=(-1*(prob_nr_graph)+prob_r_graph)*(0.73)
final_sum=sum1+sum2
if(final_sum<0):
  detection="Non-Rumour"
  message="Final Detection using joint model: Non-Rumour"
else:
  detection="Rumour"
  message="Final Detection using joint model: Rumour"
print(message.center(40,"`"))



Final Detection using joint model: Rumour


In [10]:
if(detection=="Non-Rumour"):
  exit(0)



In [11]:
exit()

In [12]:
!pip3 install torch==1.0.1 -f https://download.pytorch.org/whl/cpu/stable 
!git clone https://github.com/huggingface/torchMoji
import os
os.chdir('torchMoji')
!pip3 install -e .
#if you restart the package, the notebook risks to crash on a loop
#I did not restart and worked fine
!python3 scripts/download_weights.py

Looking in links: https://download.pytorch.org/whl/cpu/stable
fatal: destination path 'torchMoji' already exists and is not an empty directory.
Obtaining file:///content/torchMoji
  Using cached https://files.pythonhosted.org/packages/c0/3a/40967d9f5675fbb097ffec170f59c2ba19fc96373e73ad47c2cae9a30aed/numpy-1.13.1.zip
  Using cached https://files.pythonhosted.org/packages/52/67/d9ef9b5058d4a9e3f0ae641ec151790622cbeb37f157de5773358e2bf3da/scipy-0.19.1.tar.gz
  Using cached https://files.pythonhosted.org/packages/c2/38/e3b0333e661ab411545583cb24940223917fe7ffc9c68a77730dce3b10b0/scikit-learn-0.19.0.tar.gz
  Using cached https://files.pythonhosted.org/packages/09/c6/c7a477228b2162937f200ece3793bb21c0f21f66b00fc010cdeb93cf465b/text_unidecode-1.0-py2.py3-none-any.whl
  ERROR: Failed building wheel for numpy
  Running setup.py clean for numpy
  ERROR: Failed cleaning build dir for numpy
  ERROR: Failed building wheel for scipy
  Running setup.py clean for scipy
  ERROR: Failed cleaning build 

In [13]:
import numpy as np
import emoji, json
from torchmoji.global_variables import PRETRAINED_PATH, VOCAB_PATH
from torchmoji.sentence_tokenizer import SentenceTokenizer
from torchmoji.model_def import torchmoji_emojis
import statistics
from statistics import mode
from collections import Counter
import pandas as pd
demo_folder = 'ottawa'
EMOJIS = ":joy: :unamused: :weary: :sob: :heart_eyes: :pensive: :ok_hand: :blush: :heart: :smirk: :grin: :notes: :flushed: :100: :sleeping: :relieved: :relaxed: :raised_hands: :two_hearts: :expressionless: :sweat_smile: :pray: :confused: :kissing_heart: :heartbeat: :neutral_face: :information_desk_person: :disappointed: :see_no_evil: :tired_face: :v: :sunglasses: :rage: :thumbsup: :cry: :sleepy: :yum: :triumph: :hand: :mask: :clap: :eyes: :gun: :persevere: :smiling_imp: :sweat: :broken_heart: :yellow_heart: :musical_note: :speak_no_evil: :wink: :skull: :confounded: :smile: :stuck_out_tongue_winking_eye: :angry: :no_good: :muscle: :facepunch: :purple_heart: :sparkling_heart: :blue_heart: :grimacing: :sparkles:".split(' ')
stance={"support":["facepunch","sweat_smile","stuck_out_tongue_winking_eye","see_no_evil","pray","eyes","information_desk_person","wink","joy","yum","clap","sparkles","heart_eyes","ok_hand","blush","heart","flushed","sparkling_heart","100","relieved","muscle","hand","raised_hands","purple_heart","two_hearts","kissing_heart","blue_heart","v","heartbeat","sunglasses","thumbsup","smile","yellow_heart","neutral_face","pensive"],"deny":["sob","gun","triumph","confounded","grin","cry","disappointed","tired_face","rage","sleepy","smiling_imp","sweat","broken_heart","confused","skull","angry","no_good","grimacing","expressionless","unamused","speak_no_evil","smirk","weary","confounded"]}
emotions={"happy":["sparkles","speak_no_evil","sparkling_heart","blue_heart","facepunch","purple_heart","joy","muscle","musical_note","heart_eyes","ok_hand","blush","heart","yum","notes","100","relaxed","yellow_heart","raised_hands","two_hearts","kissing_heart","heartbeat","see_no_evil","thumbsup","v","sunglasses","triumph","clap","smile"],"sad":["sob","disappointed","sad","tired_face","cry","persevere","broken_heart","confounded"],"angry":["angry","no_good""skull","rage","hand","gun"],"anxious":["eyes","confused","sweat_smile","weary","pensive","flushed","pray","sweat"],"neutral":["neutral_face","expressionless","unamused","neutral","relieved","information_desk_person","notes","grimacing"],"sarcasm":["smirk","grin","sleepy","smiling_imp","wink","stuck_out_tongue_winking_eye"]}
model = torchmoji_emojis('model/pytorch_model.bin')
weightage={"support":1,"deny":1}
final_stances=[]
emotion_list=[]

with open(VOCAB_PATH, 'r') as f:
  vocabulary = json.load(f)
st = SentenceTokenizer(vocabulary, 30)
def deepmojify(sentence,top_n =5):
  def top_elements(array, k):
    ind = np.argpartition(array, -k)[-k:]
    return ind[np.argsort(array[ind])][::-1]
  tokenized, _, _ = st.tokenize_sentences([sentence])
  prob = model(tokenized)[0]
  emoji_ids = top_elements(prob, top_n)
  emojis = list(map(lambda x: EMOJIS[x], emoji_ids))
  EMO=[]
  stances=[]
  for e in emojis:
    em=e[1:len(e)-1]
    # print(em)
    for s in stance:
      if em in stance[s]:
        stances.append(s)
    for e in emotions:
      if em in emotions[e]:
        EMO.append(e)
  # print(stances)
  stances_dict=dict(Counter(stances))
  emo_dict=dict(Counter(EMO))
  print("EMOTIONS: ",emo_dict)
  wt=0
  ST=""
  weight={"happy":1,"sad":1,"anxious":1,"angry":1,"neutral":1,"sarcasm":1}
  for stance_key in stances_dict:
    if(stances_dict[stance_key]*weightage[stance_key]>wt):
      wt=stances_dict[stance_key]*weightage[stance_key]
      ST=stance_key
  # print(dict(Counter(stances)))
  wt_e=0
  for emo_key in emo_dict:
    if(emo_dict[emo_key]*weight[emo_key]>wt_e):
      wt_e=emo_dict[emo_key]*weight[emo_key]
      E=emo_key
  # print(mode(stances))
  emotion_list.append(E)
  final_stances.append(ST)
  return emoji.emojize(f"{sentence} {' '.join(emojis)}", use_aliases=True)
# text = ['Get back to work.']
# for _ in text:
#   print(deepmojify(_, top_n  = 5))


files='/content/drive/MyDrive/CAPSTONE PROJECT - PRAJNA SIRISHA SUKANYA/demo/Tweet_data/'+demo_folder
folds = sorted(os.listdir(files))
for folder in folds:
  new_path = os.path.join(files, folder)
  ids = sorted(os.listdir(new_path)) 
  newfolds = [i for i in ids if i[0] != '.']
  id_data = newfolds
  text = []
  
  final_path = os.path.join(new_path,'replies')
  tweets = sorted(os.listdir(final_path))
  for tweet in tweets:
    with open(os.path.join(final_path, tweet)) as f:
        for line in f:
            tw = json.loads(line)
            # print("TEXT:",tw['text'])
            text.append(tw['text'])
  # print("STANCES\n")
  for _ in text:
    print(deepmojify(_, top_n  = 5))
  print("\n")

# print(emotion_list)
stance_emoji_count=Counter(final_stances)
# print(stance_emoji_count)
if(stance_emoji_count["support"]>stance_emoji_count["deny"]):
  pred=1
elif(stance_emoji_count["support"]<stance_emoji_count["deny"]):
  pred=-1
else:
  pred=0



/content/torchMoji/torchmoji/model_def.py:159: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  nn.init.uniform(self.embed.weight.data, a=-0.5, b=0.5)
/content/torchMoji/torchmoji/model_def.py:161: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(t)
/content/torchMoji/torchmoji/model_def.py:163: UserWarning: nn.init.orthogonal is now deprecated in favor of nn.init.orthogonal_.
  nn.init.orthogonal(t)
/content/torchMoji/torchmoji/model_def.py:165: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(t, 0)
/content/torchMoji/torchmoji/model_def.py:167: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(self.output_layer[0].weight.data)
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1320: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.w

EMOTIONS:  {'angry': 2, 'neutral': 2, 'anxious': 1}
@20committee @BlogsofWar How come we have to pussy foot around terrorism for hours before calling what it is. 😡 😠 😑 😐 😕
EMOTIONS:  {'angry': 1, 'happy': 1, 'neutral': 3}
@20committee Pathetic. ✋ 😂 😒 😑 😐
EMOTIONS:  {'sarcasm': 3, 'happy': 1, 'neutral': 1}
@20committee @kshaidle maybe we should start revoking passports AFTER they leave... 😉 😜 😏 👍 😬
EMOTIONS:  {'happy': 1, 'anxious': 3, 'neutral': 1}
If Michael Zehaf-Bebeau had his passport confiscated doesn't it make sense to track his movements - was this done? @20committee @BlogsofWar 😂 😕 👀 😐 😳
EMOTIONS:  {'sad': 2, 'anxious': 1, 'neutral': 1, 'happy': 1}
@aenk963 @BlogsofWar yes, but CSIS can't watch that many 24/7 😢 😕 😞 😬 🙈
EMOTIONS:  {'angry': 2, 'anxious': 1, 'sarcasm': 1, 'neutral': 1}
Yes:) except in Australia there's a discussion to let 'sympathizers' reenter OZ to rehabilitate! WTF! @MarkOkanagan @20committee @kshaidle 😡 😠 😳 😜 😬
EMOTIONS:  {'angry': 2, 'happy': 1, 'sad': 2}
@2

In [14]:
!pip install npy-append-array

In [15]:
from keras.models import load_model
from collections import Counter
from npy_append_array import NpyAppendArray
import numpy as np
def predict_model_veracity():
    # x_stance= []
    # x_stance.append([x_test,emotion_list])
    x_test_ver = np.load('/content/drive/MyDrive/CAPSTONE PROJECT - PRAJNA SIRISHA SUKANYA/demo/test/test_array.npy')
    stance_lstm=predict_model_stance(emotion_list)
    model=load_model('/content/drive/MyDrive/CAPSTONE PROJECT - PRAJNA SIRISHA SUKANYA/code/Stance_Veracity/my_model_veracity.h5')
    Y_pred = model.predict_classes(x_test_ver)
    labels=[]
    true_labels=[]
    # print(Y_pred[:10])
    # for i in range(len(x_test)):
    #     print("X=%s, Predicted=%s" % (x_test[i], Y_pred[i]))
    
    # print(classification_report(y_test,Y_pred))
    preds=np.ndarray.tolist(Y_pred)
    for i in range(len(preds)):
        if(preds[i]==0):
            labels.append("true")
        elif(preds[i]==1):
            labels.append("false")
        else:
            labels.append("unverified")

    # for i in range(len(y_test)):
    #     if(y_test[i]==0):
    #         true_labels.append("true")
    #     elif(y_test[i]==1):
    #         true_labels.append("false")
    #     else:
    #        true_labels.append("unverified")
    # print(Counter(labels))
    # print(true_labels)
    predicted='Predicted Veracity'
    print(predicted.center(40,'*'))
    veracity_bilstm=max(labels,key=labels.count)
    
    
    if(pred==0):
        veracity_bilstm='Unverified Rumor'
    elif(pred==1):
        veracity_bilstm='True Rumor'
    else:
        veracity_bilstm='False Rumor'
    print(veracity_bilstm.center(40,'*'))
    print('*'*40)
    return(veracity_bilstm)
    # for i in range(len(x_test)):
    #     print(labels[i],end="\n")

def predict_model_stance(emotion_list):
    filename='/content/drive/MyDrive/CAPSTONE PROJECT - PRAJNA SIRISHA SUKANYA/demo/test/test__array.npy'
    x_test_stance = np.load(filename)
    model=load_model('/content/drive/MyDrive/CAPSTONE PROJECT - PRAJNA SIRISHA SUKANYA/code/Stance_Veracity/my_model_stance.h5')
    emo_list = []
    for em in emotion_list:
      if em=='happy':
        emo_list.append(0.0)
      elif em=='sad':
        emo_list.append(1.0)
      elif em=='anxious':
        emo_list.append(2.0)
      elif em=='angry':
        emo_list.append(3.0)
      elif em=='neutral':
        emo_list.append(4.0)
      else:
        emo_list.append(5.0)
    emo_array = np.asarray(emo_list).astype('float32')
    emo_array = emo_array[:,np.newaxis,np.newaxis]
    emo_array = np.pad(emo_array,((0,0),(2,1),(157,156)),'constant')
    # print(emo_array.shape)
    # print(x_test_stance.shape)
    # np.concatenate(x_test_stance,emo_array)

    npaa = NpyAppendArray(filename)
    npaa.append(emo_array)
    # npaa.append(arr2)
    # npaa.append(arr2)
    Y_pred=model.predict_classes(x_test_stance)
    # y_test=np.ndarray.tolist(y_test)
    Y_pred= np.ndarray.tolist(Y_pred)
    labels=[]
    # true_labels=[]
    correct=0
    for i in range(len(x_test_stance)):
        # if(int(y_test[i][1])==Y_pred[i][1]):
        #     correct+=1
        if(Y_pred[i][1]==0):
            labels.append("support")
        elif(Y_pred[i][1]==1):
            labels.append("deny")
        elif(Y_pred[i][1]==2):
            labels.append("query")
        else:
            labels.append("comment")

    # for i in range(len(y_test)):
        
    #     if(y_test[i][1]==0):
    #         true_labels.append("support")
    #     elif(y_test[i][1]==1):
    #         true_labels.append("deny")
    #     elif(y_test[i][1]==2):
    #         true_labels.append("query")
    #     else:
    #         true_labels.append("comment")

    # print("predicted")
    # for i in range(len(x_test)):
    #     print(labels[i],end="\n")
    # print("Correct predictions",correct)
    # print("Total number if tweets considered", len(x_test))
    # print("Accuracy is ", correct/len(x_test))
    # print(labels)
    # print(true_labels)
    stance_lstm_count=Counter(labels)
    if(stance_lstm_count["support"]>stance_lstm_count["deny"]):
      veracity_lstm="true"
    elif(stance_lstm_count["support"]<stance_lstm_count["deny"]):
      veracity_lstm="false"
    else:
      veracity_lstm="unverified"

    return(veracity_lstm)

    

# x_test_ver = np.load('/content/drive/MyDrive/CAPSTONE PROJECT - PRAJNA SIRISHA SUKANYA/demo/test/test_array.npy')
# y_test_ver = np.load('/content/drive/MyDrive/CAPSTONE PROJECT - PRAJNA SIRISHA SUKANYA/code/Stance_Veracity/test/labels.npy')
# print(len(x_test_ver))
# print(y_test_ver[:10])

# x_test_stance = np.load('/content/drive/MyDrive/CAPSTONE PROJECT - PRAJNA SIRISHA SUKANYA/demo/test/test_array.npy')
# y_test_stance = np.load('/content/drive/MyDrive/CAPSTONE PROJECT - PRAJNA SIRISHA SUKANYA/code/Stance_Veracity/test/fold_stance_labels.npy')
# print(len(x_test_stance))
# print(y_test_stance[:10])
# stance_lstm=predict_model_stance(x_test_stance)
veracity_bilstm=predict_model_veracity()
# print(veracity)
# veracities=[veracity_emoji,veracity_lstm, veracity_bilstm]
# print(veracities)
# veracity_final=max(veracities,key=veracities.count)
# print("FINAL PREDICTED VERACITY:",veracity_final)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


***********Predicted Veracity***********
***************True Rumor***************
****************************************
